# 使用客户服务所需的工具`ChatCompletion`端点现在包括了指定是否每次都必须调用工具的功能，方法是在参数中添加`tool_choice='required'`。这为构建封装应用程序提供了确定性的因素，因为您可以指望在每次调用时都会提供一个工具。我们将在这里演示这如何对于像客户服务这样的封闭流程非常有用，其中定义特定的退出点能够提供更多控制。笔记本最后进行了多轮评估，我们启动了一个客户GPT来模拟我们的客户，并测试我们设置的LLM客户服务代理。

In [1]:
import jsonfrom openai import OpenAIimport osclient = OpenAI()GPT_MODEL = 'gpt-4-turbo'

## 配置定义我们将定义LLM客服代理将使用的`工具`和`说明`。它将为客户面临的问题寻找正确的说明，并使用这些说明来回答客户的查询。由于这只是一个演示示例，我们将要求模型在没有外部系统提供信息时编造数值。

In [2]:
# 我们客户服务领域的语言模型将采用的沟通工具tools = [{  "type": "function",  "function": {    "name": "speak_to_user",    "description": "Use this to speak to the user to give them information and to ask for anything required for their case.",    "parameters": {      "type": "object",      "properties": {        "message": {          "type": "string",          "description": "Text of message to send to user. Can cover multiple topics."        }      },      "required": ["message"]    }  }},{  "type": "function",  "function": {    "name": "get_instructions",    "description": "Used to get instructions to deal with the user's problem.",    "parameters": {      "type": "object",      "properties": {        "problem": {          "type": "string",          "enum": ["fraud","refund","information"],          "description": """The type of problem the customer has. Can be one of:          - fraud: Required to report and resolve fraud.          - refund: Required to submit a refund request.          - information: Used for any other informational queries."""        }      },      "required": [        "problem"      ]    }  }}]# Example instructions that the customer service assistant can consult for relevant customer problemsINSTRUCTIONS = [ {"type": "fraud",                  "instructions": """• Ask the customer to describe the fraudulent activity, including the the date and items involved in the suspected fraud.• Offer the customer a refund.• Report the fraud to the security team for further investigation.• Thank the customer for contacting support and invite them to reach out with any future queries."""},                {"type": "refund",                 "instructions": """• Confirm the customer's purchase details and verify the transaction in the system.• Check the company's refund policy to ensure the request meets the criteria.• Ask the customer to provide a reason for the refund.• Submit the refund request to the accounting department.• Inform the customer of the expected time frame for the refund processing.• Thank the customer for contacting support and invite them to reach out with any future queries."""},                {"type": "information",                 "instructions": """• Greet the customer and ask how you can assist them today.• Listen carefully to the customer's query and clarify if necessary.• Provide accurate and clear information based on the customer's questions.• Offer to assist with any additional questions or provide further details if needed.• Ensure the customer is satisfied with the information provided.• Thank the customer for contacting support and invite them to reach out with any future queries.""" }]

In [3]:
assistant_system_prompt = """You are a customer service assistant. Your role is to answer user questions politely and competently.You should follow these instructions to solve the case:- Understand their problem and get the relevant instructions.- Follow the instructions to solve the customer's problem. Get their confirmation before performing a permanent operation like a refund or similar.- Help them with any other problems or close the case.Only call a tool once in a single message.If you need to fetch a piece of information from a system or document that you don't have access to, give a clear, confident answer with some dummy values."""def submit_user_message(user_query,conversation_messages=[]):    """消息处理功能，循环遍历工具调用，直到遇到需要回复的调用。一旦收到respond=True，它将返回对话消息给用户。"""    # 启动一个响应对象。当需要响应时，我们的函数会将此对象设置为True。    respond = False        user_message = {"role":"user","content": user_query}    conversation_messages.append(user_message)    print(f"User: {user_query}")    while respond is False:        # 构建一个临时消息对象，以便将对话消息添加进去        messages = [            {                "role": "system",                "content": assistant_system_prompt            }        ]        # 将对话消息添加到我们的消息调用中，以供API使用。        [messages.append(x) for x in conversation_messages]        # Make the ChatCompletion call with tool_choice='required' so we can guarantee tools will be used        response = client.chat.completions.create(model=GPT_MODEL                                                  ,messages=messages                                                  ,temperature=0                                                  ,tools=tools                                                  ,tool_choice='required'                                                 )        conversation_messages.append(response.choices[0].message)        # 执行该函数并获取更新后的 conversation_messages 对象。        # If it doesn't require a response, it will ask the assistant again.         # If not the results are returned to the user.        respond, conversation_messages = execute_function(response.choices[0].message,conversation_messages)        return conversation_messagesdef execute_function(function_calls,messages):    """包装函数，用于执行工具调用"""    for function_call in function_calls.tool_calls:            function_id = function_call.id        function_name = function_call.function.name        print(f"Calling function {function_name}")        function_arguments = json.loads(function_call.function.arguments)            if function_name == 'get_instructions':            respond = False                instruction_name = function_arguments['problem']            instructions = INSTRUCTIONS['type' == instruction_name]                messages.append(                                {                                    "tool_call_id": function_id,                                    "role": "tool",                                    "name": function_name,                                    "content": instructions['instructions'],                                }                            )            elif function_name != 'get_instructions':            respond = True                messages.append(                                {                                    "tool_call_id": function_id,                                    "role": "tool",                                    "name": function_name,                                    "content": function_arguments['message'],                                }                            )                print(f"Assistant: {function_arguments['message']}")        return (respond, messages)    

## 示例为了测试这一点，我们将为一位经历过欺诈的客户运行一个示例，看看模型如何处理。扮演用户的角色，并提供合理的下一步行动，以继续对话。

In [4]:
messages = submit_user_message("Hi, I have had an item stolen that was supposed to be delivered to me yesterday.")

User: Hi, I have had an item stolen that was supposed to be delivered to me yesterday.
Calling function get_instructions
Calling function speak_to_user
Assistant: I'm sorry to hear about the stolen item. Could you please provide me with more details about the fraudulent activity, including the date and the items involved? This information will help us to investigate the issue further and proceed with the necessary actions, including offering you a refund.


In [5]:
messages = submit_user_message("For sure, it was a shirt, it was supposed to be delivered yesterday but it never arrived.",messages)

User: For sure, it was a shirt, it was supposed to be delivered yesterday but it never arrived.
Calling function speak_to_user
Assistant: Thank you for providing the details. I will now proceed to report this incident to our security team for further investigation and arrange a refund for the stolen shirt. Please confirm if you would like me to go ahead with the refund.
Calling function speak_to_user
Assistant: Thank you for contacting us about this issue. Please don't hesitate to reach out if you have any more questions or need further assistance in the future.


In [6]:
messages = submit_user_message("Yes I would like to proceed with the refund.",messages)

User: Yes I would like to proceed with the refund.
Calling function get_instructions
Calling function speak_to_user
Assistant: Thank you for confirming. I have processed the refund for the stolen shirt. The amount should be reflected in your account within 5-7 business days. If you have any more questions or need further assistance, please feel free to contact us.


In [7]:
messages = submit_user_message("Thanks very much.",messages)

User: Thanks very much.
Calling function speak_to_user
Assistant: You're welcome! If you need any more help in the future, don't hesitate to reach out. Have a great day!


## 评估现在我们将进行一个简单的评估，其中一个GPT将假装是我们的客户。两者将互相交流，直到达成解决方案。我们将重用上面的函数，添加一个`execute_conversation`函数，其中客户GPT将继续回答。

In [8]:
customer_system_prompt = """You are a user calling in to customer service.You will talk to the agent until you have a resolution to your query.Your query is {query}.You will be presented with a conversation - provide answers for any assistant questions you receive. Here is the conversation - you are the "user" and you are speaking with the "assistant":{chat_history}If you don't know the details, respond with dummy values.Once your query is resolved, respond with "DONE" """# Initiate a bank of questions run throughquestions = ['I want to get a refund for the suit I ordered last Friday.',            'Can you tell me what your policy is for returning damaged goods?',            'Please tell me what your complaint policy is']

In [9]:
def execute_conversation(objective):    conversation_messages = []    done = False    user_query = objective    while done is False:        conversation_messages = submit_user_message(user_query,conversation_messages)        messages_string = ''        for x in conversation_messages:            if isinstance(x,dict):                if x['role'] == 'user':                    messages_string += 'User: ' + x['content'] + '\n'                elif x['role'] == 'tool':                    if x['name'] == 'speak_to_user':                        messages_string += 'Assistant: ' + x['content'] + '\n'            else:                continue        messages = [            {            "role": "system",            "content": customer_system_prompt.format(query=objective,chat_history=messages_string)            },            {            "role": "user",            "content": "Continue the chat to solve your query. Remember, you are in the user in this exchange. Do not provide User: or Assistant: in your response"            }        ]        user_response = client.chat.completions.create(model=GPT_MODEL,messages=messages,temperature=0.5)        conversation_messages.append({            "role": "user",            "content": user_response.choices[0].message.content            })        if 'DONE' in user_response.choices[0].message.content:            done = True            print("Achieved objective, closing conversation\n\n")        else:            user_query = user_response.choices[0].message.content

In [10]:
for x in questions:    execute_conversation(x)

User: I want to get a refund for the suit I ordered last Friday.
Calling function get_instructions
Calling function speak_to_user
Assistant: I understand you'd like a refund for the suit you ordered last Friday. Could you please provide more details about the issue with the suit? This will help us process your refund request accurately.
User: The suit I received is not the color I ordered. I ordered a navy blue suit, but the one I received is black.
Calling function speak_to_user
Assistant: Thank you for providing the details. I will proceed with the refund for the navy blue suit that was incorrectly sent as black. Please confirm if you would like me to go ahead with the refund.
User: Yes, please go ahead with the refund.
Calling function speak_to_user
Assistant: The refund for the incorrectly colored suit has been processed. You should see the amount credited back to your original payment method within 5-7 business days. Thank you for contacting us, and if you have any more questions 

## 结论现在，您可以通过强制使用工具来显式地控制您的LLM的行为，同时启动GPT测试人员来挑战您的LLM，并充当自动化测试用例。我们希望这让您对工具使用的一个很好的用例有所认识，并期待看到您构建的内容！